In [28]:
import mirdata
from torch.utils.data import Dataset, DataLoader
import torchaudio
import torchaudio.transforms as T
import torch.nn.functional as F
from pathlib import Path
from typing import List
import torch


In [24]:
effect_type = ["Phaser"]
root=Path("./data/egfx")
wet_files = []
dry_files = []
labels = []

In [18]:
for i, effect in enumerate(effect_type):
    for pickup in Path(root / effect).iterdir():
        wet_files += list(pickup.glob("*.wav"))
        dry_files += list(root.glob(f"Clean/{pickup.name}/**/*.wav"))
        
        labels += [i] * len(wet_files)


In [26]:
LENGTH = 2**18  # 12 seconds
ORIG_SR = 48000

In [27]:
class GuitarFXDataset(Dataset):
    def __init__(
        self,
        root: str,
        sample_rate: int,
        length: int = LENGTH,
        effect_type: List[str] = None,
    ):
        self.length = length
        self.wet_files = []
        self.dry_files = []
        self.labels = []
        self.root = Path(root)
        if effect_type is None:
            effect_type = [
                d.name for d in self.root.iterdir() if d.is_dir() and d != "Clean"
            ]
        for i, effect in enumerate(effect_type):
            for pickup in Path(self.root / effect).iterdir():
                self.wet_files += sorted(list(pickup.glob("*.wav")))
                self.dry_files += sorted(
                    list(self.root.glob(f"Clean/{pickup.name}/**/*.wav"))
                )
                self.labels += [i] * len(self.wet_files)
        print(
            f"Found {len(self.wet_files)} wet files and {len(self.dry_files)} dry files"
        )
        self.resampler = T.Resample(ORIG_SR, sample_rate)

    def __len__(self):
        return len(self.dry_files)

    def __getitem__(self, idx):
        print(idx, self.wet_files[idx], self.dry_files[idx])
        x, sr = torchaudio.load(self.wet_files[idx])
        y, sr = torchaudio.load(self.dry_files[idx])
        effect_label = self.labels[idx]

        resampled_x = self.resampler(x)
        resampled_y = self.resampler(y)
        # Pad or crop to length
        if resampled_x.shape[-1] < self.length:
            resampled_x = F.pad(resampled_x, (0, self.length - resampled_x.shape[1]))
        elif resampled_x.shape[-1] > self.length:
            resampled_x = resampled_x[:, : self.length]
        if resampled_y.shape[-1] < self.length:
            resampled_y = F.pad(resampled_y, (0, self.length - resampled_y.shape[1]))
        elif resampled_y.shape[-1] > self.length:
            resampled_y = resampled_y[:, : self.length]
        return (resampled_x, resampled_y, effect_label)


In [29]:

SAMPLE_RATE = 22050
TRAIN_SPLIT = 0.8

In [32]:
guitfx = GuitarFXDataset(
    root="./data/egfx",
    sample_rate=SAMPLE_RATE,
    effect_type=["Phaser"],
)

Found 690 wet files and 690 dry files


In [33]:
train_size = int(TRAIN_SPLIT * len(guitfx))
val_size = len(guitfx) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    guitfx, [train_size, val_size]
)
val = DataLoader(val_dataset, batch_size=2)


In [37]:
print(val_dataset.indices)

[560, 150, 218, 404, 292, 509, 10, 315, 554, 6, 169, 116, 601, 309, 280, 510, 559, 197, 613, 424, 500, 460, 273, 467, 190, 534, 642, 112, 635, 283, 217, 7, 679, 526, 73, 102, 134, 263, 449, 142, 215, 154, 181, 378, 425, 278, 208, 58, 323, 210, 388, 363, 249, 57, 479, 79, 508, 429, 237, 390, 435, 62, 254, 528, 614, 311, 680, 61, 374, 668, 373, 594, 9, 677, 188, 2, 91, 633, 549, 257, 170, 183, 465, 502, 244, 664, 632, 356, 581, 145, 81, 85, 232, 250, 571, 118, 319, 308, 536, 592, 607, 566, 609, 302, 576, 354, 35, 493, 593, 437, 636, 495, 506, 153, 638, 164, 229, 456, 34, 518, 381, 322, 304, 565, 52, 499, 66, 39, 220, 38, 111, 454, 267, 98, 563, 585, 121, 391]


In [38]:
for v in val:
    print(v)

[tensor([[[0.0482, 0.0772, 0.0682,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0092, 0.0138, 0.0139,  ..., 0.0000, 0.0000, 0.0000]]]), tensor([[[-0.0007, -0.0009,  0.0074,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0007,  0.0036,  0.0064,  ...,  0.0000,  0.0000,  0.0000]]]), tensor([0, 0])]
[tensor([[[0.0027, 0.0050, 0.0063,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0043, 0.0077, 0.0084,  ..., 0.0000, 0.0000, 0.0000]]]), tensor([[[0.0007, 0.0023, 0.0026,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0005, 0.0023, 0.0034,  ..., 0.0000, 0.0000, 0.0000]]]), tensor([0, 0])]
[tensor([[[0.0008, 0.0017, 0.0056,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0008, 0.0016, 0.0016,  ..., 0.0000, 0.0000, 0.0000]]]), tensor([[[0.0007, 0.0022, 0.0028,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0003, 0.0009, 0.0011,  ..., 0.0000, 0.0000, 0.0000]]]), tensor([0, 0])]
[tensor([[[ 0.0071,  0.0107,  0.0078,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0043,  0.0011, -0.0055,  ...,  0.0000,  